# How to write a scikit-learn compatible estimator

## Adrin Jalali
### @adrinjalali, scikit-learn

### Intro to scikit-learn

A machine learning library implementing statistical machine learning methods.

__Out of scope:__
- Neural Networks (except a simple MLP implementation)
- GPU

__Important components in the API:__

- estimators (transformers and predictors)
- scorers
- meta-estimators
    - pipeline
    - grid search

``` python
# A common workflow includes a pipeline once the data is loaded.
# We usually preprocess the data and prepare it for the
# final classifier or regressor.
# We call each step an "estimator", the preprocessing steps which
# augment the data "transformers", and the final step a classifier
# or a regressor.
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

# Each step can be tuned with many hyper-parameters, and we want to
# find the best hyper-parameter set for the given dataset.
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__max_iter': (20,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
}

# find the best parameters for both the feature extraction and the
# classifier, we use a grid search.
grid_search = GridSearchCV(pipeline, parameters)
```

## Why a custom estimator?

- scikit-learn doesn't include all algorithms, and it has a very high bar for including one. You can test your new or modified algorithm as a custom estimator.
- The library does not include methods which are appropriate only for a small set o use-cases, and if you happen to work on one of those problems, you might need to develop your own estimator to tackle the specific issues you have.
- You may want to add some steps before or after running each step, in which case you can write a meta-estimator wrapping around the steps you usually would have in a pipeline.

### Basic API

- `fit (X, y, **kwargs)`
- `predict(X)` (`predict_proba` and `decision_function`)
- `transform(X)`
- `score(X, y[, sample_weight])`


## Our Custom Transformer

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import (
    check_array, check_is_fitted, check_X_y)
import numpy as np
import pandas as pd
import spacy

class MyTransformer(TransformerMixin, BaseEstimator):
    def fit(self, X, y, **fit_params):
        self.model_ = spacy.load("en_core_web_md")
        return self

    def transform(self, X, y):
        check_is_fitted(self)
        docs = self.model_.pipe(X)
        return np.array(map(lambda x: x.vector, docs))

In [21]:
from sklearn.base import ClassifierMixin
from sklearn.linear_model import LogisticRegression

class MyClassifier(ClassifierMixin, BeseEstimator):
    def __init__(self, C=1.0, random_state=0):
        self.C = C
        self.random_state = random_state
    
    def fit(self, X, y, **fit_params):
        X, y = check_X_y(X, y)
        self.estimator_ = LogisticRegression(C=self.C,
                                             random_state=self.random_state)
        self.classes_ = self.estimator_.classes_
    
    def predict(self, X):
        check_is_fitted(self)


NameError: name 'BeseEstimator' is not defined

In [31]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
import numpy as np

X, y = make_classification(n_classes=3, n_informative=3, random_state=0)
probs = LogisticRegression(random_state=0).fit(X, y).predict_proba(X[:3])
res = np.argmax(probs, axis=1)
mask = np.max(probs, axis=1) < .9
res[~mask] = -1
res

array([ 1, -1,  0])

``` python
def check_is_fitted(estimator, attributes=None, msg=None, all_or_any=all):
    """Perform is_fitted validation for estimator.

    """

def check_array(array, accept_sparse=False, accept_large_sparse=True,
                dtype="numeric", order=None, copy=False, force_all_finite=True,
                ensure_2d=True, allow_nd=False, ensure_min_samples=1,
                ensure_min_features=1, estimator=None):

    """Input validation on an array, list, sparse matrix or similar.

    By default, the input is checked to be a non-empty 2D array containing
    only finite values. If the dtype of the array is object, attempt
    converting to float, raising on failure.
    """
    
def check_X_y(X, y, accept_sparse=False, accept_large_sparse=True,
              dtype="numeric", order=None, copy=False, force_all_finite=True,
              ensure_2d=True, allow_nd=False, multi_output=False,
              ensure_min_samples=1, ensure_min_features=1, y_numeric=False,
              estimator=None):
    """Input validation for standard estimators.

    Checks X and y for consistent length, enforces X to be 2D and y 1D. By
    default, X is checked to be non-empty and containing only finite values.
    Standard input checks are also applied to y, such as checking that y
    does not have np.nan or np.inf targets. For multi-label y, set
    multi_output=True to allow 2D and sparse y. If the dtype of X is
    object, attempt converting to float, raising on failure.

```

## Run Common Tests on Our Classifier

In [2]:
from sklearn.utils.estimator_checks import parametrize_with_checks

@parametrize_with_checks([MyClassifier])
def test_sklearn_compatible_estimator(estimator, check):
    check(estimator)

In [3]:
import ipytest
ipytest.config(rewrite_asserts=True, magics=True)
__file__ = 'custom_estimators.ipynb'

logger.setLevel(logging.INFO)

ipytest.run()

============================================================================ test session starts ============================================================================
platform linux -- Python 3.8.1, pytest-5.3.3, py-1.8.1, pluggy-0.13.1
rootdir: /home/adrin/Documents/talks/sklearn-estimator-fosdem-2020
collected 40 items

custom_estimators.py ........................................                                                                                                         [100%]

============================================================================ 40 passed in 0.24s =============================================================================


### Fit the Model

In [4]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

logger.setLevel(logging.DEBUG)

X, y = make_classification(n_samples=1000, n_informative=10)
X_train, X_test, y_train, y_test = train_test_split(X, y)

clf = MyClassifier().fit(X_train, y_train)

2020-01-31 16:56:06,382 C: 1
2020-01-31 16:56:06,382 fitting on 750 samples
2020-01-31 16:56:06,397 fit end/


### Get a Score

In [5]:
clf.score(X_test, y_test)

2020-01-31 16:56:06,405 predicting on 250 samples


0.916

## In a Pipeline

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

pipe = make_pipeline(SelectKBest(k=2), MyClassifier(C=.1))
pipe.fit(X_train, y_train).score(X_test, y_test)

2020-01-31 16:56:06,432 C: 0.1
2020-01-31 16:56:06,432 fitting on 750 samples
2020-01-31 16:56:06,443 fit end/
2020-01-31 16:56:06,445 predicting on 250 samples


0.76

## The Pipeline in a Grid Search

In [7]:
logger.setLevel(logging.INFO)

param_grid = {'selectkbest__k': [1, 2, 5, 10, 20],
              'myclassifier__C': [0.1, 1, 100]}
gs = GridSearchCV(pipe, param_grid=param_grid).fit(X_train, y_train)
gs.score(X_test, y_test)

0.916

In [8]:
gs.best_estimator_

Pipeline(memory=None,
         steps=[('selectkbest',
                 SelectKBest(k=20,
                             score_func=<function f_classif at 0x7f3e9f5e0280>)),
                ('myclassifier', MyClassifier(C=1))],
         verbose=False)

## Conventions

- `fit` should only get sample aligned data in `fit_params`
    - everything else should go through `__init__`
- `__init__` doesn't set anything other than the parameters passed to it
- `obj.attr` is set through `__init__` and `set_params`
- `obj.attr_` is set during fit and counts as public API
- `obj._attr` is private

## Estimator Tags

``` python
_DEFAULT_TAGS = {
    'non_deterministic': False,
    'requires_positive_X': False,
    'requires_positive_y': False,
    'X_types': ['2darray'],
    'poor_score': False,
    'no_validation': False,
    'multioutput': False,
    "allow_nan": False,
    'stateless': False,
    'multilabel': False,
    '_skip_test': False,
    'multioutput_only': False,
    'binary_only': False,
    'requires_fit': True}
```

You can change them with:

``` python
class MyMultiOutputEstimator(BaseEstimator):

    def _more_tags(self):
        return {'multioutput_only': True,
                'non_deterministic': True}
```

## Upcoming Features

- `n_features_in_`[#16112](https://github.com/scikit-learn/scikit-learn/pull/16112), `n_features_out_`[#14241](https://github.com/scikit-learn/scikit-learn/pull/14241)
- `feature_names_in_`, `feature_names_out_` [SLEP#7](https://github.com/scikit-learn/enhancement_proposals/pull/17), [SLEP#8](https://github.com/scikit-learn/enhancement_proposals/pull/18), [SLEP#12](https://github.com/scikit-learn/enhancement_proposals/pull/25)
- sample/feature/data properties (through `_request_props`?) [SLEP#6](https://github.com/scikit-learn/enhancement_proposals/pull/16), [#16079](https://github.com/scikit-learn/scikit-learn/pull/16079)

## More Details/Further Reading

- [https://scikit-learn.org/dev/developers/develop.html](https://scikit-learn.org/stable/developers/develop.html)
- `sklearn/base.py`
- `sklearn/metaestimators.py`
- `sklearn/utils/validation.py`

## Questions?
### Thank you!